## 0. Import library

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1.Import data

In [51]:
train=pd.read_csv('titanic_traning.csv')
#train.drop(columns='ID',inplace=True)
train.head()

,ID,pclass,sex,age,sibsp,parch,fare,embarked,survived
0,1,3,male,20.0,0,0,4.0,C,0
1,2,3,female,18.0,0,0,7.2,C,1
2,3,2,male,NaN,0,0,15.6,C,0
3,4,2,male,NaN,0,0,0.0,S,0
4,5,3,female,2.0,3,2,27.9,S,0


In [52]:
train.columns[1:]

Index(['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked',
       'survived'],
      dtype='object')

In [53]:
#Name of features
feat_list=train.columns[1:]
print(feat_list)
#Create a dict that contains all possible values for the features that have limited unique values 
factor={'sex':['male', 'female'],
        'embarked':['C', 'S', 'Q'],
        'pclass':np.arange(1,4),
        'sibsp':np.arange(9),
        'parch':np.arange(10),
        'survived':[0,1]
       }

Index(['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked',
       'survived'],
      dtype='object')


## 2. Function to find the missing values and inconsistency

In [54]:
## Function to find number of missing values
def check_missing_value(feat):
    nomv=0 #Number of missing values
    #check if there is any missing values
    if train[feat].isnull().any():
        #if yes, count the number of missing value
        mis_val=train[feat][train[feat].isnull()]
        print('\nThe records that have missing value in the feature',feat,':')
        print(train[train[feat].isnull()])
        nomv=len(mis_val)
    return nomv
    
#example of function
print(check_missing_value('fare'))

## Function to find the inconsistency
def check_inconsistency(feat):
    noic=0 #Number of inconsistent values
    pv=train[feat].unique()
    #print(pv)
    if np.isin(feat,list(factor.keys())):
        # deal with features that have limited unique values
        iv=pv[np.isin(pv,factor[feat])==False].copy()
        if len(iv)!=0:
            print('\nFor feature',feat,', the cases that have inconsistent values are:')
            for val in iv:
                subset=train[feat][train[feat]==val]
                print(train[train[feat]==val])
                noic+=len(subset)
    else:
        # deal with features that may have unlimited unique values like age and fare
        # Nan will not be counted because we already count it in the missing values checking
        pv=pv[~np.isnan(pv)]
        #check if all unique values is a positive number
        for val in pv:
            if val<0: 
                subset=train[feat][train[feat]==val]
                noic+=len(subset)
    return noic

#example of function
print(check_inconsistency('sex'))    


The records that have missing value in the feature fare :
      ID  pclass     sex   age  sibsp  parch  fare embarked  survived
8      9       2    male  44.0      1      0   NaN        S         0
27    28       1    male  41.0      1      0   NaN        S         0
62    63       3    male  34.0      1      1   NaN        S         0
81    82       1  female  31.0      0      0   NaN        C         1
88    89       1  female  40.0      0      0   NaN        S         1
89    90       3    male  25.0      0      0   NaN        C         0
135  136       3  female   9.0      4      2   NaN        S         0
165  166       3    male  60.5      0      0   NaN        S         0
908  909       1  female  30.0      0      0   NaN        C         1
9

For feature sex , the cases that have inconsistent values are:
      ID  pclass   sex   age  sibsp  parch  fare embarked  survived
5      6       1  Male  45.0      0      0  29.7        C         1
756  757       3  Male  27.0      0    

## 2. Main program


In [55]:
N=len(train.pclass)
problem_table=pd.DataFrame(index=feat_list)
mv=[]
iv=[]
for feat in feat_list:
    mv.append(check_missing_value(feat))
    iv.append(check_inconsistency(feat))
problem_table['Missing Values']=mv
problem_table['Percentage of MV']=[round(100*x/N,2) for x in mv ]
problem_table['Inconsistent Values']=iv
problem_table['Percentage of IV']=[round(100*x/N,2) for x in iv ]
problem_table


For feature sex , the cases that have inconsistent values are:
      ID  pclass   sex   age  sibsp  parch  fare embarked  survived
5      6       1  Male  45.0      0      0  29.7        C         1
756  757       3  Male  27.0      0      0   7.8        S         1
      ID  pclass     sex   age  sibsp  parch  fare embarked  survived
160  161       3  Female  24.0      0      2  16.7        S         1
169  170       3  Female   NaN      0      0   8.1        S         0

The records that have missing value in the feature age :
      ID  pclass     sex  age  sibsp  parch   fare    embarked  survived
2      3       2    male  NaN      0      0   15.6           C         0
3      4       2    male  NaN      0      0    0.0           S         0
11    12       3  female  NaN      0      0    7.7           Q         1
13    14       3  female  NaN      2      0   23.3           Q         1
18    19       1    male  NaN      0      0   26.0           S         0
21    22       3    male  

,Missing Values,Percentage of MV,Inconsistent Values,Percentage of IV
pclass,0,0.00,0,0.00
sex,0,0.00,4,0.44
age,188,20.52,0,0.00
sibsp,0,0.00,0,0.00
parch,0,0.00,0,0.00
fare,9,0.98,0,0.00
embarked,0,0.00,3,0.33
survived,0,0.00,0,0.00


### 2.1. Handle the missing data

In [56]:
def fillna_by_mean(feat):
    #Find the mean of the feature
    mean=np.mean(train[feat][-train[feat].isnull()])
    train[feat][train[feat].isnull()]=round(mean,2)
fillna_by_mean('age')
fillna_by_mean('fare')

C:\Users\Duong Hung\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


### 2.2. Handle the inconsistent data

In [57]:
# For sex feature
train['sex'][train['sex']=='Male']='male'
train['sex'][train['sex']=='Female']='female'
# For embarked feature
train['embarked'][train['embarked']=='Queenstown']='Q'


C:\Users\Duong Hung\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Duong Hung\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Duong Hung\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


## 3. Store the cleaned data into a file

In [60]:
train.to_csv('cleaned_titanic_traning.csv',index=False)

In [59]:
#Check the cleaned data to make sure all problems are solved ( No printing output means good)
for feat in feat_list:
    mv.append(check_missing_value(feat))
    iv.append(check_inconsistency(feat))
